## 동아일보

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import os
import re
from datetime import datetime
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './donga'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

categories = {
    '정치': 'https://www.donga.com/news/Politics',
    '경제': 'https://www.donga.com/news/Economy',
    '국제': 'https://www.donga.com/news/Inter',
    '사회_사건': 'https://www.donga.com/news/Society/Event', # 앞 분기 기준
    '경제_과학': 'https://www.donga.com/news/It/List', # 앞 분기 기준
    '사회': 'https://www.donga.com/news/Society',
    '스포츠': 'https://www.donga.com/news/Sports',
}

result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

# 현재 날짜의 오전 8시 (시간대는 +09:00 기준)
today = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0, tzinfo=datetime.now().astimezone().tzinfo)

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        page_category = soup.find('ol', class_='breadcrumb').find('li', class_='breadcrumb_item').find('a').get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('section', {'class': 'head_group'}).find('h1').get_text(strip=True)
    except AttributeError:
        title = 'N/A'
        
    try:
        date_button = soup.find('button', {'aria-controls': 'dateInfo'})
        time_span = date_button.find('span', {'aria-hidden': 'true'})
        time = time_span.get_text(strip=True)  # "2024-09-30 21:44" 형태
        time = time.replace(" ", "T") + ":00+09:00"  # ISO 8601 형식으로 변환 (예: 2024-09-30T21:44:00+09:00)
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        # 광고, 이미지 등 제외하고 텍스트만 추출
        content_section = soup.find('section', class_='news_view')
        for tag in content_section(['figure', 'script', 'div', 'ul']):  # 불필요한 태그 제거
            tag.decompose()
        content = content_section.get_text(separator="\n", strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'source_url': article_url, 'content': content, 'category': result_category, 'source_created_at': time, 'press_id': 2, 'press_name' : '동아일보'}

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = article_data.get('category')
    file_name = date + "-" + str(file_idx) + '.json'
    result_dir = output_dir + "/" + category + "/" + date + "-" + str(file_idx)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    file_path = os.path.join(result_dir, 'source-' + file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('article', class_='news_card')
    if category not in ['사회_사건', '경제_과학']:
        cards.reverse()
    for card in cards:
        if article_count >= 4:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            logging.error("result is None");
            continue

        if result.get('category') not in ['accident', 'science']:
            # 'source_created_at'를 datetime 객체로 변환
            article_time = datetime.strptime(result.get('source_created_at'), '%Y-%m-%dT%H:%M:%S%z')
            # 오늘 오전 8시 이후가 아니면 continue
            if article_time < today:
                logging.info(f"{article_link} 은 오늘 오전 8시 이전 기사이므로 생략합니다.")
                continue
        
        print(result)
        article_count += 1
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()
        save_article_as_json(result, date, article_count)

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 정치 (https://www.donga.com/news/Politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Society/article/all/20241008/130175675/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/article/all/20241008/130175675/1 HTTP/11" 200 None
INFO:root:page_category: 사회
INFO:root:category: 정치
INFO:root:https://www.donga.com/news/Society/article/all/20241008/130175675/1 는 해당 카테고리가 아니어서 생략합니다. 
ERROR:root:result is None
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241008/130175850/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/202

{'title': '명태균·김대남, 10일 행안위 국감 증인 불출석 통보…“검찰 수사 중”', 'source_url': 'https://www.donga.com/news/Politics/article/all/20241008/130175850/1', 'content': '국회 행정안전위원회가 국정감사 증인으로 채택한 명태균 씨와 김대남 전 SGI서울보증 상근감사위원이 8일 불출석 의사를 밝혔다. 명 씨는 윤석열 대통령의 부인 김건희 여사 ‘공천 개입 의혹’의 핵심 관련자다. 김 전 감사는 대통령실 비서관 직무대리 출신으로, 국민의힘 전당대회를 앞두고 당권주자였던 한동훈 대표에 대한 공격을 사주했다는 의혹을 받고 있다.\n행안위에 따르면 명 씨와 김 전 감사는 이날 국회에 국감 불출석 이유서를 제출했다. 명 씨는 이유서에 “국정감사 및 조사에 관한 법률에 의거하여 검찰 수사 중이라 출석이 어렵다”고 밝혔다. 김 전 감사는 형사소송법과 국회 증감법(국회에서의 증언‧감정 등에 관한 법)에 따라 불출석한다고 이유서를 냈다.\n앞서 행안위는 더불어민주당 주도로 김 여사의 4월 총선 공천 개입 의혹을 살펴보겠다며 김 전 감사와 명 씨, 명 씨에게 공천 청탁을 했다는 의혹을 받는 국민의힘 김영선 전 의원 등을 10일 열리는 중앙선거관리위원회 대상 국감에 증인으로 부르기로 했다.\n명 씨가 연루된 김 전 의원의 공천 청탁 의혹에 대해선 창원지검 형사4부가 수사를 진행하고 있다. 고위공직자범죄수사처도 시민단체 ‘사법정의 바로세우기 시민행동’이 윤 대통령 부부와 명 씨 등을 공천 개입 의혹과 관련해 직권남용 권리행사방해·공직선거법 위반·청탁금지법 위반 혐의로 고발한 사건을 수사하고 있다.\n또 김 전 감사는 ‘김 여사가 (공천관리위원이던) 국민의힘 이철규 의원을 통해 4월 총선 공천에 개입했다’는 취지의 주장을 편 녹취록이 공개돼 이 의원으로부터 허위사실 유포 및 명예훼손 혐의로 고소를 당했다.', 'category': 'politics', 'source_created_at': 

INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:Article saved to ./donga/politics/2024-10-08-2\source-2024-10-08-2.json
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241008/130175986/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241008/130175986/1 HTTP/11" 200 None
INFO:root:page_category: 정치


{'title': '김영호 “北, 회색지대 도발 가능성…인명 피해시 강경 대응”', 'source_url': 'https://www.donga.com/news/Politics/article/all/20241008/130175904/1', 'content': '김영호 통일부 장관은 8일 북한의 쓰레기 풍선 살포가 빈번한 것과 관련해 “회색지대 도발을 단행할 가능성이 점점 높아지고 있다”면서 우리의 인명 피해가 발생할 시 “강경하게 대응해야 한다”라고 강조했다.\n김 장관은 이날 오전 국회에서 열린 외교통일위원회 국정감사에서 윤상현 국민의힘 의원의 북한의 쓰레기 및 오물풍선 살포와 관련한 질의에 “시간이 갈수록 숫자도 많아지고 낙하 지점도 자기들이 의도하는 대로 보내고 있다”면서 이같이 말했다.\n이날 윤 의원은 “결국 북한이 원하는 것은 풍선에 기폭장치까지 넣어 풍향이니, 거리니, 무게니 이런 것을 전부 데이터로 축적하려는 것”이라며 “비대칭적 군사 위협 요인으로 활용하기 위해 하나의 실험을 진행하는 것”이라고 주장했다.\n김 장관은 우리가 쓰레기 풍선 살포에 대해 군사적 조치를 취할 수 있는 명확한 ‘레드라인’을 설정해 북한에 알려야 한다는 윤 의원의 요구에 “(풍선에) 화학물질을 넣어서 보내거나 우리 국민의 인명 피해가 날 경우는 명백한 도발”이라면서 “우리 정부가 적절하게, 강하게 대응해야 한다”라고 강조했다.\n(서울=뉴스1)', 'category': 'politics', 'source_created_at': '2024-10-08T15:18:00+09:00', 'press_id': 2, 'press_name': '동아일보'}


INFO:root:category: 정치
INFO:root:Article saved to ./donga/politics/2024-10-08-3\source-2024-10-08-3.json
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241008/130176145/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241008/130176145/1 HTTP/11" 200 None
INFO:root:page_category: 정치


...

{'title': '“4년간 지옥이었다”…학창시절 왕따 고백한 성시경과 박서준', 'source_url': 'https://www.donga.com/news/Society/article/all/20241008/130176008/2', 'content': '가수 성시경(45)과 배우 박서준(35)이 학창 시절 따돌림을 당했다고 털어놨다.\n유튜브 채널 ‘성시경’에는 지난 7일 ‘처음 만난 서준이한테 반해버렸습니다’라는 제목의 영상이 게재됐다.\n영상에서 박서준은 “어릴 때는 진짜 소심하고 낯가림도 엄청 심했다. 너무 쑥스러워서 식당에서 주문도 못 했다”며 “그런 성격이 너무 싫었던 건 중학교 3학년 때였다. 어디 가서 말도 못 하고 제 자신이 너무 바보 같았다”고 말문을 열었다.\n이에 성시경은 “허우대도 멀쩡하고 키도 컸을 텐데 친구들 사이에서 왕따는 없지 않았나”고 물었고, 박서준은 “초등학교 땐 왕따도 좀 당했다. 중학교 때도 자연스럽게 조용한 부류 중 한 명이었다”고 말했다.\n성시경은 “요즘 왕따 논란이 많다. 나도 처음 얘기하는데 (초등학교) 1학년 때부터 4학년 때까지 지옥이었다”며 “(가해자들이) ‘끝나고 남아’라고 하는데 난 누굴 때리는 성격도 아니고 싸움도 싫었다. 근데 지나고 나니까 누굴 때린 거보다 다행인 거 같다”고 말했다.\n그러면서 “(왕따가) 얼마나 지옥 같은지 난 정확히 기억하고 있다. 애들이 참 순수한데 순수한 만큼 너무 순수하게 못됐다”고 덧붙였다.\n박서준은 “결국에는 자존감과 자신감의 문제인 것 같다”며 “그때는 당연히 내가 ‘난 별거 아니니까’라며 자신감도 많이 떨어진 아이였는데 연기를 시작하면서 자신감도 생기고 얘기도 조금 할 수 있게 됐다”고 전했다.', 'category': 'society', 'source_created_at': '2024-10-08T15:36:00+09:00', 'press_id': 2, 'press_name': '동아일보'}
{'title': '한국도자재단, ‘2024경기도자비엔날레 제3차 라운드

INFO:root:category (base_url): 스포츠 (https://www.donga.com/news/Sports)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241008/130172363/2
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241008/130172363/2 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:https://www.donga.com/news/Sports/article/all/20241008/130172363/2 은 오늘 오전 8시 이전 기사이므로 생략합니다.
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241008/130173476/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241008/130173476/1 HTTP/11" 200 

{'title': '손흥민 부상·황희찬 부진…더 중요해진 이강인 왼발', 'source_url': 'https://www.donga.com/news/Sports/article/all/20241008/130173476/1', 'content': '10일 오후 11시 요르단 원정 경기\n경기력 좋은 이강인 어깨 무거워\n한국 축구의 현재이자 미래로 평가받는 이강인(23·파리 생제르맹)의 중요성이 더 커졌다.\n10월 A매치에서 ‘주장’ 손흥민(32·토트넘)의 부상 공백과 ‘주포’ 황희찬(28·울버햄튼)의 부진까지 덮을 수 있는 활약을 펼쳐야 한다.\n홍명보 감독이 이끄는 한국 남자 축구 대표팀은 오는 10일 오후 11시(한국시각) 요르단 암만의 암만 국제경기장에서 요르단과 ‘2026 국제축구연맹(FIFA) 북중미 월드컵’ 아시아 지역 3차 예선 3차전을 치른다.\n요르단전 이후에는 한국으로 귀국, 15일 오후 8시 용인미르스타디움에서 이라크와 4차전으로 10월 A매치를 마무리한다.\n현재 다득점에서 요르단에 밀려 B조 2위에 있는 홍명보호는 이번 2연전 전승을 통해 선두 도약을 꾀한다.\n목표 달성이 쉽진 않을 전망이다.\n지난 2월 카타르에서 막을 내린 ‘2023 아시아축구연맹(AFC) 아시안컵’ 4강전에서 한국에 0-2 충격패 탈락을 안겼던 요르단을 원정에서 상대하는 데다, 주장이자 핵심 공격 자원인 손흥민이 부상으로 낙마했다.\n홍 감독은 지난달 30일 발표한 10월 A매치 명단 26인에 손흥민을 포함했으나, 왼쪽 허벅지 부상이 길어지면서 결국 홍현석(25·마인츠)을 대체 발탁 결정했다.\n꾸준하게 한국 축구의 핵심 자원이자, 지난 9월 첫선을 보인 홍명보호에서 에이스로 맹활약했던 손흥민이 빠지는 건 뼈아프다.\n설상가상 손흥민의 빈자리를 대체할 수 있는 황희찬마저 부진하다.\n황희찬은 지난 시즌 잉글랜드 프로축구 프리미어리그(EPL)에서만 12골을 기록하면서 빅 리그에서 좋은 활약을 펼쳤으나, 이번 시즌 아직 마수걸이 득점을 넣지 못했다.\n홍 감독은 “소

DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241008/130174057/1 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./donga/sports/2024-10-08-3\source-2024-10-08-3.json
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241008/130174377/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241008/130174377/1 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./donga/sports/2024-10-08-4\source-2024-10-08-4.json


{'title': 'FIFA, 황희찬에게 인종차별 발언한 선수 ‘10경기 출장정지’', 'source_url': 'https://www.donga.com/news/Sports/article/all/20241008/130174057/1', 'content': '지난 7월 울버햄튼과 코모의 친선전 중 발생\n절반은 집행 유예, 사회봉사와 교육도 받아야\n축구 국가대표 공격수 황희찬에게 인종차별적 발언을 했던 마르코 쿠르토(이탈리아)가 국제축구연맹(FIFA)으로부터 10경기 출전 정지 징계를 받았다.\n영국 매체 ‘BBC’는 8일(한국시각) “코모(이탈리아 세리에A) 소속 쿠르토는 울버햄튼(EPL)과의 프리시즌 경기 중 황희찬에게 인종차별을 가한 혐의로 FIFA로부터 10경기 출전 정지 징계를 받았다. 이 중 5경기는 2년 동안 집행이 유예된다”고 전했다.\n매체에 따르면 FIFA 대변인은 “쿠르토는 인종차별적 행동을 범한 혐의가 있다고 판단돼 10경기 출전 정지 징계를 받았다”고 밝히며 “절반은 2년 동안 집행이 유예되며, 선수는 지역 사회 봉사와 FIFA 승인 단체 교육을 받아야 한다”고 덧붙였다.\n지난 7월 울버햄튼과 코모의 평가전 당시 황희찬은 쿠르토에게 인종차별적 모욕을 들었고, 팀 동료 다니엘 포덴세가 주먹을 날려 퇴장을 당하는 등 소동이 발생했다.\n게리 오닐 감독은 “차니(Channy, 황희찬 애칭)는 정말 실망스러운 인종차별적 발언을 들었다. 경기를 그만두고 싶은지 물었지만 그는 계속 뛰길 원했다”고 설명했다.\n코모는 “당사자(쿠르토)와 대화했는데, 황희찬이 동료들에게 ‘차니’라고 불리는 걸 보고 ‘자신이 재키 챈(성룡)이라고 생각하는 것 같다. 무시하라’고 말했다고 한다”고 해명했다.\n이어 “의도적으로 깎아내리는 말을 한 적이 없다. 우리는 일부 울버햄튼 선수들의 반응으로 인해 이 사건이 지나치게 과장된 것에 실망했다”며 적반하장식 태도를 보였다.\n당시 울버햄튼은 UEFA 주관 경기가 아니라 공식 항의할 수 없다고 비판했지만, 대한축구협회가

## 중앙일보

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import os
import re
from datetime import datetime
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './joongang'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

categories = {
    '정치': 'https://www.joongang.co.kr/politics',
    '경제': 'https://www.joongang.co.kr/money',
    '국제': 'https://www.joongang.co.kr/world',
    '사회_사건': 'https://www.joongang.co.kr/society/accident', # 앞 분기 기준
    '경제_과학': 'https://www.joongang.co.kr/money/science', # 앞 분기 기준
    '사회': 'https://www.joongang.co.kr/society',
    '스포츠': 'https://www.joongang.co.kr/sports',
}


result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

# 현재 날짜의 오전 8시 (시간대는 +09:00 기준)
today = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0, tzinfo=datetime.now().astimezone().tzinfo)

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:

        page_category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        title = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'source_url': article_url, 'content': content, 'category': result_category, 'source_created_at': time, 'press_id': 1, 'press_name': '중앙일보'}

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = article_data.get('category')
    file_name = date + "-" + str(file_idx) + '.json'
    result_dir = output_dir + "/" + category + "/" + date + "-" + str(file_idx)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    file_path = os.path.join(result_dir, 'source-' + file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('li', class_='card')
    cards.reverse()
    for card in cards:
        if article_count >= 4:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue

        if result.get('category') not in ['accident', 'science']:
            # 'source_created_at'를 datetime 객체로 변환
            article_time = datetime.strptime(result.get('source_created_at'), '%Y-%m-%dT%H:%M:%S%z')
            # 오늘 오전 8시 이후가 아니면 continue
            if article_time < today:
                logging.info(f"{article_link} 은 오늘 오전 8시 이전 기사이므로 생략합니다.")
                continue

        print(result)
        article_count += 1
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()
        save_article_as_json(result, date, article_count)

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 정치 (https://www.joongang.co.kr/politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /politics HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282695
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282695 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:https://www.joongang.co.kr/article/25282695 은 오늘 오전 8시 이전 기사이므로 생략합니다.
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282701
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282701 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:https://www.joongang.co.kr/articl

{'title': '김정은, 푸틴 72세 생일 축하 축전…"가장 친근한 나의 동지"', 'source_url': 'https://www.joongang.co.kr/article/25282768', 'content': '블라디미르 푸틴 러시아 대통령(왼쪽)과 김정은 북한 국무위원장이 지난 19일 평양 에서 정상회담 뒤 서명한 조약을 들어 보이고 있다. 타스=연합뉴스김정은 북한 국무위원장이 72세 생일을 맞은 블라디미르 푸틴 러시아 대통령에게 축하를 전했다.김 위원장은 푸틴 대통령에게 축전을 보내 "당신의 탄생일을 맞으며 충심으로 되는 따뜻한 축하의 인사를 보낸다"고 밝혔다고 조선중앙통신이 8일 보도했다.김 위원장은 "앞으로도 계속될 우리들 사이의 상봉과 동지적 유대는 새로운 전면적 발전궤도 위에 올라선 조로(북러) 친선과 전략적 협조 관계의 만년 기틀을 더욱 공고히 하고 지역과 세계의 평화, 국제적 정의를 수호하는 데 적극 이바지하게 될 것"이라고 강조했다.그는 푸틴 대통령의 지난 6월 평양 방문을 계기로 북러 관계가 "자주와 정의 실현을 공동이념으로 하는 불패의 동맹관계, 백년대계의 전략적 관계"로 승격됐다며 "우리들 사이의 친분을 더욱 두터이 하던 그날을 감회 깊게 회억하고 있다"고 덧붙였다.아울러 푸틴 대통령이 러시아를 "승리와 영광의 한길로 인도하리라 믿는다"며 "러시아 군대와 인민의 정의의 위업에 대한 우리의 전폭적이며 사심 없는 지지와 연대성을 재확언한다"고 밝혔다. 이는 북한이 우크라이나를 침공한 러시아의 편에 있다는 점을 재차 강조한 것으로 보인다.김 위원장은 축전의 시작과 끝에서 푸틴 대통령을 "가장 친근한 동지"라고 불렀다. 이는 북한과 중국 수교 75주년이었던 지난 6일 김 위원장이 시진핑(習近平) 중국 국가주석에게 보낸 축전에서의 표현과 대비된다.김 위원장은 2019년 북중 수교 70주년 때만 해도 시 주석에게 보낸 축전에서 시 주석을 \'존경하는 총서기 동지\'라고 칭했다. 하지만 올해는 \'존경하는\'이라는 표현을 사용하지 않아 북중 관

INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:Article saved to ./joongang/politics/2024-10-08-2\source-2024-10-08-2.json
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282785
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282785 HTTP/11" 200 None
INFO:root:page_category: 정치


{'title': '김정은 "핵무기 사용 배제 안 해…그 무엇도 한국 지켜주지 못해"', 'source_url': 'https://www.joongang.co.kr/article/25282769', 'content': '북한 노동당 기관지 노동신문은 8일 김정은 국무위원장이 전날인 7일 창립 60주년을 맞이한 김정은국방종합대학을 축하방문하고 연설을 했다고 보도했다. 김정은은 이날 연설에서 ″군사초강국, 핵강국을 향한 발걸음이 더 빨라질 것″이라고 말했다. 노동신문=뉴스1김정은 북한 국무위원장이 핵무기사용을 언급하며 한미동맹을 위협했다.북한 노동당 기관지 노동신문의 8일 보도에 따르면, 김 위원장은 7일 김정은국방종합대학 창립 60주년 연설에서 “적들이 우리 국가를 반대하는 무력사용을 기도한다면 공화국무력은 모든 공격을 주저없이 사용할 것”이라며 “여기에는 핵무기사용이 배제되지 않는다”고 말했다.김정은은 이어 “그런 상황에서 생존에 희망을 거는 것은 부질없는 일이며 행운도, 그 무슨 신의 보호도 대한민국을 지켜주지 못할 것”이라고 거듭 위협했다.그는 “한미동맹을 핵에 기반한 동맹으로 변이시키고 무력증강에 열을 올리면서 광적으로 벌려놓는 미제와 괴뢰들의 전쟁소동과 도발적 행태는 언제든 조선반도에서 힘의 균형이 깨질 수 있는 위험성을 내포하고 있다”고 말했다. 이어 “조선반도(한반도)에서 전략적 힘의 균형 파괴는 곧 전쟁을 의미한다”면서 “적을 항상 억제하고 정세를 관리할 수 있는 물리적 힘을 가져야 한다는 우리의 자위국방 건설 논리는 바늘 들어갈 틈도 없이 완벽하고 정당하다”고 주장했다.그러면서 “군사초강국, 핵강국에로 향한 우리의 발걸음은 더 빨라질 것”이라며 “한미군사동맹이 괴뢰들 스스로가 광고하는 것처럼 핵동맹으로 완전히 변이된 현시점에서 우리 국가의 핵대응태세는 더더욱 한계를 모르는 높이에서 완비되어야 한다”고 했다. 또 “적들이 핵동맹을 무기로 힘의 우위를 차지하고 전략적 판세를 뒤집어놓으려고 발악할수록 우리는 국방과학과 공업의 계속적인 도약을 이룩하고 

INFO:root:category: 정치
INFO:root:Article saved to ./joongang/politics/2024-10-08-3\source-2024-10-08-3.json
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282790
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282790 HTTP/11" 200 None
INFO:root:page_category: 정치


...

{'title': '여자 축구대표팀 새 사령탑에 신상우 김천 상무 코치 낙점', 'source_url': 'https://www.joongang.co.kr/article/25282795', 'content': '여자 축구대표팀 감독으로 낙점된 신상우 김천 코치. 사진 김천 상무여자 축구대표팀 새 사령탑으로 신상우 프로축구 K리그1 김천상무 코치가 낙점됐다.8일 최영일 대한축구협회 부회장에 따르면 신상우 코치가 여자 대표팀 감독으로 선임됐다. 이사회도 통과했고, 공식 발표 시점을 고민하는 단계다. 일본과 평가전이 오는 26일 열릴 예정이라서 빠르게 선임 절차를 마치고, 코치진과 선수단을 구성해야 하는 상황이다.\xa0여자 대표팀 감독은 지난 6월 콜린 벨 감독과 계약을 종료한 이후 공석이다.한국 정서에 맞고 유소년 축구부터 다 챙길 수 있는 국내 지도자를 염두에 뒀던\xa0축구협회는 신 감독을 두고 약 한 달간 기술 철학과 (여자축구의) 방향성 등을 철저하게 검증하는 선임 절차를 거쳤다. 올 시즌 김천이 K리그1에서 좋은 성적을 낸 것도 가산점이 됐다. 김천은 현재 울산 HD에 이어 리그 2위를 달리며 우승을 노리고 있다.2022년부터 김천에서 코치로 일한 신 감독은 여자축구 전문가다.\xa02015년 여자 실업축구 WK리그 보은상무 수석 코치를 지낸 후 2017년에는 이천 대교 지휘봉을 잡았다.\xa0WK리그 명문 구단이었던 대교가 2017시즌을 끝으로 해체되자, 2018년부터는 창녕WFC로 둥지를 옮겨 초대 감독으로서 팀을 이끌었다.피주영 기자 akapj@joongang.co.kr', 'category': 'sports', 'source_created_at': '2024-10-08T11:17:17+09:00', 'press_id': 1, 'press_name': '중앙일보'}


INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./joongang/sports/2024-10-08-3\source-2024-10-08-3.json
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282831
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282831 HTTP/11" 200 None


{'title': '황희찬에 "재키 챈" 조롱한 선수, 인종차별로 \'10경기 출전정지\'', 'source_url': 'https://www.joongang.co.kr/article/25282798', 'content': '올 시즌을 앞두고 연습경기 중 인종차별적 발언을 들은 황희찬. AFP=연합뉴스축구대표팀 공격수 황희찬(울버햄프턴)에게 연습 경기 중 인종차별적 발언을 한 마르코 쿠르토(체세나)가 국제축구연맹(FIFA)으로부터 10경기 출전 정지 징계를 받았다.7일(현지시간) 영국 스포츠 매체 스카이스포츠에 따르면 쿠르토는 코모 1907(이탈리아)에서 뛰던 지난 7월 울버햄프턴(잉글랜드)과의 프리 시즌 연습 경기 후반 23분 황희찬을 향해 인종차별적 발언을 했다는 혐의가 인정돼 FIFA가 징계를 확정했다. FIFA는 "쿠르토가 차별적인 발언을 한 데 책임이 인정돼 10경기 출전 정지 징계를 내린다"고 밝혔다. 그러면서 "이 가운데 절반인 5경기에 대해서는 2년간 출전 정지 조치의 집행이 유예된다"고 설명했다.사건이 일어난 시점에 코모 구단은 쿠르토가 동료 수비수에게 \'무시해, 황희찬은 스스로를 재키 챈(홍콩 스타 성룡)이라 생각한다\'고 말한 걸로 파악했다고 밝혔다. 이 경기 당시\xa0울버햄프턴에서 뛴 다니엘 포덴세(알샤바브)가 동료 황희찬을 향한 인종차별적 발언에 분노해 쿠르토에게 주먹을 날린 뒤 퇴장을 당했다.\xa0게리 오닐 울버햄프턴 감독은 황희찬에게 "계속 경기를 뛰어도 괜찮겠냐"고 물었다. 황희찬은 경기를 끝까지 소화했다.경기 후 코모는 황희찬이 울버햄프턴 동료들에게 \'차니\'라고 불리는 걸 듣고 쿠르토가 이 같은 발언을 했을 거라고 해명했다.\xa0쿠르토는 황희찬을 향한 인종차별적 발언에 따른 논란이 불거지자 문제의 연습 경기가 열린 후 1주 만에 코모를 떠나 이탈리아 2부 리그 팀인 체세나로 임대 이적했다. 울버햄프턴의 축구 분야 최고 책임자 맷 와일드는 "이런 징계는 축구에서 인종차별이 허용되면 안 된다는 분명한 메시지를 전달한다"며 

INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./joongang/sports/2024-10-08-4\source-2024-10-08-4.json


{'title': '히딩크도 페예노르트 에이스 황인범 칭찬 "그라운드서 리더처럼 행동"', 'source_url': 'https://www.joongang.co.kr/article/25282831', 'content': '히딩크의 칭찬을 받은 황인범. AFP=연합뉴스한국 축구대표팀을 이끌고 2002 한일월드컵에서 4강 신화를 일궜던 명장 거스 히딩크(네덜란드) 감독이 자국 리그에서 맹활약하는 황인범(페예노르트)의 실력을 높이 평가했다.히딩크 감독은 지난 6일(현지시간) 네덜란드 스포츠 방송 스튜디오 풋발에 출연해 "황인범을 잘 알지 못하지만, 유럽에서 경험이 많은 선수고, 리더처럼 행동하는 모습이 긍정적"이라고 칭찬했다.\xa0특히 히딩크 감독은 그라운드에서 황인범의 적극적인 모습에 높은 점수를 줬다. 그는\xa0"나는 한국 사람들과 일해봤다. 대부분 매우 얌전한 성격이지만 황인범이 동료들에게 몇 차례 손으로 지시하는 걸 봤다"고 설명했다.\'중원 사령관\' 황인범은 지난 6일 트벤테와의 2024~25시즌 네덜란드 에레디비시 8라운드 홈 경기에서 결승 골을 터뜨리며 팀의 2-1 승리를 이끌었다. 페예노르트는 개막 7경기 무패(3승 4무)로 리그 5위를 달렸다. 황인범은 페예노르트가 1-0으로 앞선 전반 43분 왼발 중거리슛으로 골을 터뜨렸다. 시즌 1호 골이자 페예노르트 데뷔골이었다. 그는 이 경기 최우수선수(MOM)에도 선정됐다.황인범은 지난달 3일 세르비아 츠르베나즈베즈다에서 네덜란드 페예노르트로 이적했다. 이미 밴쿠버(미국), 루빈 카잔(러시아), 올림피아코스(그리스), 즈베즈다 등 다양한 리그를 경험한 그는 페예노르트 이적과 동시에 주전 자리를 꿰찼다. 주 무기인 날카로운 패스로 빼어난 활약을 펼친 끝에 구단 선정 ‘9월의 선수’로 뽑히는 등 이적하자마자 팀의 에이스로 자리 잡았다.황인범은 지난 7일 스포츠 매체 ESPN으로부터 8라운드에서 최고의 활약을 보여준 선수로도 선정됐다. ESPN 네덜란드판은 "황인범이 이렇게 짧은 기간에 존재감을 드러낸 건 

## 한국경제

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

categories = {
    '경제': 'https://www.hankyung.com/economy',
    '정치': 'https://www.hankyung.com/politics'
}


def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
      
        # 해당 페이지 카테고리
        page_category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        # 찾고있는 카테고리
        logging.info(f"category: {category}")

        category = category.split('_')
        
        # 페이지 카테고리 != 찾고있는 카테고리 -> 스킵
        if page_category not in category:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
        
    except AttributeError:
        # 유효하지 않은 url 스킵
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        headline = soup.find('h3', {'class': 'next-fit'}).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('span', {'class': 'txt-date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('p', {'class': 'lead'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    
    return {'title': headline, 'time': time, 'content': content}


def check_for_new_articles(base_url, category):
    
    # 결과 받아오기
    response = requests.get(base_url, headers=header, allow_redirects=False)
    print(f"Status Code: {response.status_code}")
    print(f"Location (Redirect URL): {response.headers.get('Location')}")
    
    # 받은 결과를 BeautifulSoup로 전달
    soup = BeautifulSoup(response.text, 'html.parser')

    # 현재 가져온 기사 갯수 초기화
    article_count = 0
    
    # 해당 태그와 일치하는 내용 가져오기
    articles = soup.find_all('h3', class_='news-fit')
    
    # 최신 기사를 가져오기 위하 정렬 수행
    articles.reverse()
    
    # 가져온 card를 탐색
    for article in articles:
        
        # 기사를 3개 추출한 경우 종료
        if article_count >= 3:
            break

        # 기사 url 추출
        link_tag = article.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        # 기사 추출
        result = scrap_article(article_link, category)

        # 추출된 기사가 있는 경우에만 다음 로직을 수행
        if result:

            print(result)
            # 카운팅
            article_count += 1
    

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 경제 (https://www.hankyung.com/economy)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.hankyung.com:443
DEBUG:urllib3.connectionpool:https://www.hankyung.com:443 "GET /economy HTTP/11" 200 None


Status Code: 200
Location (Redirect URL): None


INFO:root:category (base_url): 정치 (https://www.hankyung.com/politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.hankyung.com:443
DEBUG:urllib3.connectionpool:https://www.hankyung.com:443 "GET /politics HTTP/11" 200 None


Status Code: 200
Location (Redirect URL): None
